In [1]:
import time
import requests
import csv
import numpy as np
from scipy.stats import iqr
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from nsepython import *
from bsedata.bse import BSE
bse = BSE(update_codes = True)

In [2]:
# Session ID and CSRF token obtained from somewhere (e.g., through authentication)
session_id = '6odudjtarsmhbs9bcw0ogkpwbgem1xde'
csrf_token = 'OGwEq4RNQN2KReVvHdSR6b0UGHrSFtqt'

# Construct headers with session ID and CSRF token
headers = {
    'Cookie': f'sessionid={session_id}',  # Assuming the session ID is passed via cookie
    'X-CSRFToken': csrf_token  # Assuming the CSRF token is passed in a header
}

# Initialize an empty list to store the symbols
symbols = []

# Loop through pages 1 to 5
for page in range(1, 4):  # Pages 1 to 5
    url = f'https://www.screener.in/ipo/recent/?page={page}'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the response text with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='data-table')
        if table:
            rows = table.find_all('tr')
            for row in rows[1:]:  # Skipping header row
                cols = row.find_all('td')
                if cols and len(cols) > 0:
                    symbol = str(cols[0].find('a')).split('/')[2]  # Extract symbol
                    symbols.append(symbol)
    else:
        print(f"Failed to fetch page {page}. Status code: {response.status_code}")

symbols.remove('id')
# Print the collected symbols
print("Collected Symbols:")
print(symbols)

In [3]:
def download_stock_data(symbol):
    if any(char.isdigit() for char in symbol):
        return yf.download(bse.getQuote(symbol)['securityID'] + ".BO", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
    else:
        return yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
    
    
def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data
    

def plot_stock_data(symbol, data):
    plot_data = data
    
    # Setup subplot layout
    figure = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    
    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'), row=1, col=1)
    
    # Plot moving averages
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'), row=1, col=1)

    # Plot volume with color-coded bars (green for positive, red for negative days)
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True, row=1, col=1)

    # Update axes
    figure.update_yaxes(range=[plot_data['Close'].min()*0.8, plot_data['Close'].max()*1.2], row=1, col=1)
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # Hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True, row=1, col=1)
    figure.update_yaxes(visible=False, secondary_y=True, row=1, col=1)

    return figure


def process_stocks(all_symbols):
    figure_html = open(f'/Users/rakeshbali/Imperial/Stock Research/Shortlisted/IPO/IPO.html', 'w')
    for symbol in all_symbols:
        try:
            data = download_stock_data(symbol)
            data = calculate_technical_indicators(data)
            figure = plot_stock_data(symbol, data)
            figure_html.write(figure.to_html(full_html=False))
        except Exception as e:
            e 
    figure_html.close()


if __name__ == "__main__":
    process_stocks(symbols)
    print('Execution Completed!!!!')